# Video Capture and Prediction of 1s Frame Sequences

<br>
<br>
Modes: <br>
1. Capture - Saves concatenated frames to a file.  <br>
2. Predict - Attempts to predict sequences
<br>
<br>

### For reference:
<img src='https://i.pinimg.com/originals/fa/bb/70/fabb7087b0cffe30530e7df9ec1d0b88.png' width="400" height="500"></img>
<br>
<br>

In [ ]:
# !pip install -Uqq fastbook
# import fastbook
# fastbook.setup_book()

# from fastbook import *
from fastai.vision.widgets import *
import fastai.vision
import fastai
from fastai.learner import *

import numpy as np
import cv2 as cv

import os

<br>
<br>

## Load the model

In [ ]:
path = '../models'
# learn_inf = load_learner(path/'200922-0716-FALL-lrfoc4-ulrfoc2-ulrfoc2')
learn_inf = load_learner(f'{path}/200925-1745-RESNET101-384px-FONLY-ft3f3.pkl', cpu=False)

# learn_inf = load_learner(path/'200921-0600-FALL-lrfoc4-ulrfoc6-F-lrfoc4-ulrfoc6')

<br>
<br>

## Function definitions

In [ ]:
def setup_camera() -> cv.VideoCapture:
    '''
    Set up the camera source
    '''
#     cap = cv.VideoCapture(0)
    # cap = cv.VideoCapture('http://127.0.0.1:4747/video')
    cap = cv.VideoCapture('http://10.0.0.75:4747/video')
    # cap = cv.VideoCapture('Droidcam')
    # cap = cv.VideoCapture('http://10.0.0.145:4747/video')
    
    print(f'Autofocus status: {cap.set(cv.CAP_PROP_AUTOFOCUS, 0)}')
    print(f'Manual focus to shortest distance: {cap.set(cv.CAP_PROP_FOCUS, 0)}')

    # Check if camera opened successfully
    if not cap.isOpened():
        print("Error opening video stream or file!")
    
    return cap




def close_camera():
    '''
    Close all capture devices and destroy open capture windows
    '''
    cap.release()
    cv.destroyAllWindows()
    
    


def print_stats():
    '''
    Print some basic stats to stdout
    '''
    
    print(f'Frame shape = {frame.shape}')
    print(f'Total number of Frames = {nframe}')
    print(f'Number of frames processed = {n_proc_frames // PROC_NTH_FRAME}')
    
    
    
# @TODO: Factor out the directory creation logic - slow/redundant    
def save_images(frame, prframe):
    '''
    write small and large images to jpeg
    '''
    symbol_dir = SYMBOL
    
    #seperate the directory path
    datadir_small = f'../data/{maindir}-S/{symbol_dir}'
    datadir_large = f'../data/{maindir}-L/{symbol_dir}'

    # build the entire directory structure if it doesn't exist
    if not os.path.isdir(datadir_small):
        os.makedirs(datadir_small)
        
    if not os.path.isdir(datadir_large):
        os.makedirs(datadir_large)
    
    # create a directory+filename template
    ftemplate_small = f'{datadir_small}/{file_prefix}-{n_proc_frames}.jpg'
    ftemplate_large = f'{datadir_large}/{file_prefix}-{n_proc_frames}.jpg'
    
    # write the image to a file
#     cv.imwrite(ftemplate_small, prframe)
    cv.imwrite(ftemplate_large, frame)
    
    frame = cv.putText(frame, f'Saved Frame #{n_proc_frames}', 
                   org=(20,40), fontFace=cv.FONT_HERSHEY_PLAIN, 
                   fontScale=2, color=(0,255,255), thickness=2,
                   lineType=cv.LINE_AA) 
    
    # Display the frame with the writing on it
    cv.imshow('What the Camera Sees:',frame)
    
    
    
    


<br>
<br>

## Open the video camera, loop for every Frame, massage the image and make a prediction.  

#### This is the main code cel.  It has 2 purposes. When you hit the spacebar, it will either:
1. Make an ASL alphabet translation or 
2. If SAVE_IMAGES_TO_FILE is True, it will save an image to the data dir 

In [ ]:
PROC_NTH_FRAME = 4  # Skip every N-1 frames - increase this if your computer lags
SAVE_IMAGES_TO_FILE = False  # flag if we want to capture the image to disk

maindir = 'frank-ledlights'  # these are image saving parameters dir/file
file_prefix = 'oldwebcam'

n_proc_frames, nframe = 0,0  # number of frames

# declare here to widen scope
pred, probs, pred_idx = [0], [0], 0
frame, prframe = None, None

if SAVE_IMAGES_TO_FILE:
    SYMBOL = input('Enter Symbol of interest: ')[0]
    
cap = setup_camera()


# "Game" Loop ... for every frame
while(cap.isOpened()):
    nframe += 1
    ret, frame = cap.read()  # Capture frame
    wait_ret = cv.waitKey(2)  # key char value if any

    # Break out of the loop if the user presses 'Q'
    if wait_ret & 0xFF == ord('q'):
        print_stats()
        break
    
    # only process the rest if the capture was successful
    if not ret:
        print("Can't read frame from capture source!")
        break  # break out of loop to let the camera close

    # crop the original 640x480 image down to a centered square 480x480
    frame = frame[:, 80:-80]

    # mirror the image horizontally
    frame = cv.flip(frame, 1)


    # process only if the space key is hit and
    # only process every PROC_NTH_FRAME frames 
    if (nframe % PROC_NTH_FRAME == 0):  # and wait_ret & 0xFF == ord(' ')  
        n_proc_frames +=1  # num processed frames

        # resize the image to 128x128 after "zooming in"
#         prframe = cv.resize(frame[60:-60, 60:-60], (128,128))

        # this version for resnet34 w/ 300px image
#         prframe = cv.resize(frame[60:-60, 60:-60], (300,300))

        # this version for resnet34 w/ 300px image
        prframe = cv.resize(frame[60:-60, 60:-60], (384,384))
        
        # Display the frame that we pass through the predictor
        cv.imshow('What the Predictor Sees:', prframe)

        if SAVE_IMAGES_TO_FILE :
            save_images(frame, prframe)
            continue  # no need to predict

        # create the prediction
        pred,pred_idx,probs = learn_inf.predict(prframe)


    # write the prediction as text on the frame
    if probs[pred_idx] > 0.75:
        frame = cv.putText(frame, f'Predict: {pred}, prob: {probs[pred_idx]:.02f}', 
                           org=(20,40), fontFace=cv.FONT_HERSHEY_PLAIN, 
                           fontScale=2, color=(0,255,255), thickness=2,
                           lineType=cv.LINE_AA) 

    # Display the original frame
    cv.imshow('What the Camera Sees:',frame)

# Release the capture object
close_camera()


In [ ]:
cap.release()
cv.destroyAllWindows()                     

<br>
<br>
<br>
<br>
<br>


# The rest of this file is for reference only.  

<br>
<br>
<br>



<br>
<br>
<br>
<br>
<br>
<br>
<br>
<br>


# Snippets

```python
# rotate the image
frame = cv.rotate(frame, cv.ROTATE_90_COUNTERCLOCKWISE)
```